<a href="https://www.kaggle.com/code/ipatra/music-recommendation-system?scriptVersionId=183786291" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a id="1"></a>

<h1 style="background-color:Yellow;font-family:newtimeroman;font-size:350%;text-align:center;border-radius: 20px 20px;color:Black;"> Music Recommendation Sytem
</h1>

## Overview of Music Recommendation Systems
Music recommendation systems are sophisticated algorithms and models designed to suggest music tracks to users based on various criteria. The primary objective of these systems is to enhance user experience by providing personalized music recommendations that align with the user's tastes and preferences. Here's a comprehensive overview of the components, methodologies, and technologies involved in music recommendation systems:

### Key Components

1. Data Collection:

   * User Data: Includes user demographics, listening history, search queries, and interaction patterns.
   * Music Data: Encompasses metadata such as genre, artist, album, release year, and audio features like tempo, key, and rhythm.

2. Preprocessing:

   * Data Cleaning: Handling missing values, duplicates, and irrelevant data.
   * Feature Extraction: Extracting relevant features from audio tracks using tools like Mel-Frequency Cepstral Coefficients (MFCCs), chroma     features, and spectral contrast.
   

3. Recommendation Algorithms:

   * Collaborative Filtering:
       * User-Based: Recommends tracks based on the preferences of similar users.
       * Item-Based: Suggests tracks similar to those a user has liked in the past.
       * Content-Based Filtering: Recommends tracks based on the similarity of music attributes to those of the tracks a user has previously enjoyed.
       * Hybrid Models: Combine collaborative and content-based methods to improve recommendation accuracy.
       

4. Model Training and Evaluation:

   * Training: Utilizing machine learning techniques such as matrix factorization, neural networks, and clustering.
   * Evaluation Metrics: Precision, recall, F1 score, Mean Average Precision (MAP), and user satisfaction scores.
   

5. User Interface and Feedback Loop:

   * Interface Design: Ensuring the recommendations are easily accessible and engaging to users.
   * Feedback Mechanism: Collecting user feedback to refine and improve the recommendation model continuously.

In [68]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kkbox-music-recommendation-challenge/test.csv.7z
/kaggle/input/kkbox-music-recommendation-challenge/train.csv.7z
/kaggle/input/kkbox-music-recommendation-challenge/songs.csv.7z
/kaggle/input/kkbox-music-recommendation-challenge/members.csv.7z
/kaggle/input/kkbox-music-recommendation-challenge/song_extra_info.csv.7z
/kaggle/input/kkbox-music-recommendation-challenge/sample_submission.csv.7z


# Import necessary libraries

In [69]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

# Load the dataset

In [70]:
!pip install datatable

In [71]:
!pip install pyunpack patool

import os , sys
from pyunpack.cli import Archive

os.system('apt-get install p7zip')
print(os.getcwd()) #/kaggle/working
import datatable as dt

directory = '/kaggle/working/'
Archive('/kaggle/input/kkbox-music-recommendation-challenge/train.csv.7z').extractall(directory)
Archive('/kaggle/input/kkbox-music-recommendation-challenge/test.csv.7z').extractall(directory)
Archive('/kaggle/input/kkbox-music-recommendation-challenge/songs.csv.7z').extractall(directory)
Archive('/kaggle/input/kkbox-music-recommendation-challenge/members.csv.7z').extractall(directory)
Archive('/kaggle/input/kkbox-music-recommendation-challenge/song_extra_info.csv.7z').extractall(directory)

#sys.exit("Error message")
train_df = reduce_mem_usage(dt.fread('./train.csv').to_pandas())
test_df = reduce_mem_usage(dt.fread('./test.csv').to_pandas())
songs_df = reduce_mem_usage(dt.fread('./songs.csv').to_pandas()) #'composer', 'lyricist'
members_df = reduce_mem_usage(dt.fread('./members.csv').to_pandas())

Reading package lists...
Building dependency tree...
Reading state information...
p7zip is already the newest version (16.02+dfsg-7build1).
0 upgraded, 0 newly installed, 0 to remove and 64 not upgraded.
/kaggle/working
Memory usage of dataframe is 288.46 MB
Memory usage after optimization is: 89.45 MB
Decreased by 69.0%
Memory usage of dataframe is 107.29 MB
Memory usage after optimization is: 42.17 MB
Decreased by 60.7%
Memory usage of dataframe is 113.88 MB
Memory usage after optimization is: 159.83 MB
Decreased by -40.4%
Memory usage of dataframe is 1.18 MB
Memory usage after optimization is: 1.83 MB
Decreased by -54.8%


Assuming train_df, test_df, songs_df, and members_df are already defined

In [72]:
print('Data loading completed!')
print(train_df.shape, test_df.shape, songs_df.shape, members_df.shape)

Data loading completed!
(7377418, 6) (2556790, 6) (2296320, 7) (34403, 7)


Displaying the first few rows of the 'train_df' DataFrame to get a quick overview of the data

In [73]:
train_df.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1.0
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1.0
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1.0
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1.0
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1.0


The code `train_df.isnull().sum()` is used to count the number of missing values (null values) in each column of the DataFrame called `train_df`.

In [74]:
train_df.isnull().sum()

msno                  0
song_id               0
source_system_tab     0
source_screen_name    0
source_type           0
target                0
dtype: int64

Displaying the first few rows of the 'test_df' DataFrame to inspect the data structure and values

In [75]:
test_df.head()

,id,msno,song_id,source_system_tab,source_screen_name,source_type
0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,local-library
1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,Local playlist more,local-library
2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,discover,,song-based-playlist
3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,radio,Radio,radio
4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=,radio,Radio,radio


# Preparing and improving the members dataset by cleaning and enhancing its features

Count the number of missing values in each column of the 'songs_df' DataFrame

In [76]:
songs_df.isnull().sum()

song_id        0
song_length    0
genre_ids      0
artist_name    0
composer       0
lyricist       0
language       1
dtype: int64

Displaying the first few rows of the 'songs_df' DataFrame to quickly inspect its contents

In [77]:
songs_df.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,,,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


Count the occurrences of each unique value in the 'genre_ids' column of the 'songs_df' DataFrame

In [78]:
songs_df.genre_ids.value_counts()

genre_ids
465         567911
958         176349
2022        168870
1609        166457
2122        139938
             ...  
465|1007         1
1138|430         1
458|698          1
1208|139         1
423|531          1
Name: count, Length: 1046, dtype: int64

Splitting the values in the 'genre_ids' column of the 'songs_df' DataFrame using the '|' delimiter

In [79]:
df_idx = songs_df["genre_ids"].str.split('|')

In [80]:
import time

start = time.time()

songs_df = pd.concat([songs_df.drop('genre_ids', axis = 1 ), df_idx.apply(pd.Series)], axis=1)

end = time.time()
print(end - start)

613.1359078884125


# Completing the one empty space

In [81]:
songs_df[songs_df['composer'].str.startswith('Korean')].language.value_counts()

language
31.0    125
3.0      27
52.0      2
Name: count, dtype: int64

In [82]:
songs_df.loc[songs_df['language'].isnull(),'language'] = 31.0

# Preparing and improving the members data by cleaning and enhancing its features

Displaying the first few rows of the members dataframe to get a quick overview of the data

In [83]:
members_df.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,,7,20110820,20170920
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,,7,20150628,20170622
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,,4,20160411,20170712
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,,9,20150906,20150907
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,,4,20170126,20170613


In [84]:
members_df['registration_init_time'] = pd.to_datetime(members_df['registration_init_time'], format='%Y%m%d')
members_df['reg_year'] = members_df['registration_init_time'].dt.year
members_df['reg_month'] = members_df['registration_init_time'].dt.month
members_df['reg_day'] = members_df['registration_init_time'].dt.day

# expiration_date
members_df['expiration_date'] = pd.to_datetime(members_df['expiration_date'],  format='%Y%m%d')
members_df['exp_year'] = members_df['expiration_date'].dt.year
members_df['exp_month'] = members_df['expiration_date'].dt.month
members_df['exp_day'] = members_df['expiration_date'].dt.day

This code snippet removes the specified columns ('registration_init_time' and 'expiration_date') from the 'members_df' DataFrame.

In [85]:
members_df.drop(['registration_init_time','expiration_date'] , axis = 1 , inplace= True)

Displaying the first few rows of the members DataFrame to inspect its structure and data

In [86]:
members_df.head()

,msno,city,bd,gender,registered_via,reg_year,reg_month,reg_day,exp_year,exp_month,exp_day
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,,7,2011,8,20,2017,9,20
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,,7,2015,6,28,2017,6,22
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,,4,2016,4,11,2017,7,12
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,,9,2015,9,6,2015,9,7
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,,4,2017,1,26,2017,6,13


# Removing unnecessary information from 'bd' (birthdate) and 'gender' in the data cleaning process

In [87]:
members_df['bd'] = members_df['bd'].apply(lambda x : 0 if x > 100 else abs(x) )

In [88]:
members_df['gender'] = members_df['gender'].astype("category").cat.codes

In [89]:
train_age = members_df[members_df['bd'] != 0].copy() 
y_train_age  = train_age['bd']
train_age.drop(['msno' ,'bd','gender'],axis = 1 , inplace = True )
test_age =  members_df[members_df['bd'] == 0].copy() 
y_test_age  = test_age['bd']
test_age.drop(['msno' ,'bd','gender'],axis = 1 , inplace = True )

In [90]:
train_age.head()

,city,registered_via,reg_year,reg_month,reg_day,exp_year,exp_month,exp_day
5,13,9,2012,7,3,2017,10,6
10,12,9,2006,5,31,2017,8,16
13,15,3,2015,9,15,2017,10,19
15,13,3,2014,4,10,2017,4,27
16,13,9,2007,3,30,2017,6,12


# Sorting or categorizing individuals based on their age and gender, especially when there are many instances where the values are 0

In [91]:
from lightgbm import LGBMClassifier , LGBMRegressor
from catboost import CatBoostClassifier ,CatBoostRegressor

from sklearn.model_selection import GridSearchCV , cross_val_score
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_squared_error as MSE

In [92]:
model = CatBoostRegressor(verbose = 0)
model.fit(train_age.values, y_train_age.values.ravel())
cv_score = np.sqrt(- cross_val_score(model, train_age.values,y_train_age.values.ravel(), cv=3 ,scoring='neg_mean_squared_error').mean())
cv_score

8.873272710866614

In [93]:
preds_age = model.predict(test_age.values)
preds_age = preds_age.round()
preds_age

array([38., 25., 24., ..., 28., 28., 27.])

In [94]:
members_df['bd'].value_counts()

bd
0     19954
22      751
27      750
24      740
26      719
      ...  
90        1
2         1
70        1
3         1
10        1
Name: count, Length: 80, dtype: int64

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming 'bd' is a column in your 'members_df' DataFrame
sns.set(style="whitegrid")  # Optional: Set a style for the plot
fig, ax = plt.subplots(figsize=(12, 8))

# Plotting the histogram with red color
sns.histplot(x='bd', data=members_df, bins=10, color='blue', ax=ax)

# Adding a title to the plot
ax.set_title('Histogram of bd', fontsize=16)

plt.show()


In [96]:
age_nulls_indexes = members_df[members_df['bd'] == 0].index
j = 0
for i in age_nulls_indexes : 
    members_df.loc[i , 'bd'] = preds_age[j]
    j += 1

In [97]:
members_df['bd'].value_counts()

bd
26    3537
27    3410
28    2884
25    2813
29    2671
      ... 
90       1
76       1
2        1
97       1
10       1
Name: count, Length: 79, dtype: int64

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming members_df is your DataFrame and 'bd' is a column in it
sns.set(style="whitegrid")  # Optional: Set the style if needed

# Create the histogram with red color
sns.histplot(x='bd', data=members_df, bins=10, color='Light-Red')

# Set the title of the figure
plt.title('Histogram of bd')

# Show the plot
plt.show()


In [99]:
train_gender =  members_df[members_df['gender'] != 0].copy() 
y_train_gender  = train_gender['gender']
train_gender.drop(['msno','gender'],axis = 1 , inplace = True )
test_gender =  members_df[members_df['gender'] == 0].copy() 
y_test_gender  = test_gender['gender']
test_gender.drop(['msno' ,'gender'],axis = 1 , inplace = True )

In [100]:
model = CatBoostClassifier(verbose = 0 )
model.fit(train_gender.values, y_train_gender.values.ravel())
cv_score = cross_val_score(model, train_gender.values,y_train_gender.values.ravel(), cv=3 ,scoring='accuracy').mean()
cv_score

0.5258948136836681

In [101]:
predictions = model.predict(test_gender.values)

In [102]:
members_df['gender'].value_counts()

gender
0    19902
2     7405
1     7096
Name: count, dtype: int64

In [103]:
gender_nulls_indexes = members_df[members_df['gender'] == 0].index
j = 0
for i in gender_nulls_indexes : 
    members_df.loc[i , 'gender'] = predictions[j]
    j += 1

In [104]:
members_df['gender'].value_counts()

gender
1    17802
2    16601
Name: count, dtype: int64

In [105]:
members_df.head()

,msno,city,bd,gender,registered_via,reg_year,reg_month,reg_day,exp_year,exp_month,exp_day
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,38,1,7,2011,8,20,2017,9,20
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,25,1,7,2015,6,28,2017,6,22
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,24,1,4,2016,4,11,2017,7,12
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,33,1,9,2015,9,6,2015,9,7
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,25,1,4,2017,1,26,2017,6,13


# Combining the train and test datasets to get them ready for preparation steps

In [106]:
ntrain = train_df.shape[0]
ntest = test_df.shape[0]
y_train = train_df['target'].values
test_IDs = test_df['msno'].copy()
all_data = pd.concat((train_df.drop('target',axis =1), test_df)).reset_index(drop=True)
print("all_data size is : {}".format(all_data.shape))

all_data size is : (9934208, 6)


In [107]:
all_data.drop('id', axis =1 , inplace = True)

In [108]:
all_data = all_data.merge(members_df, on='msno', how='left')

all_data = all_data.merge(songs_df, on = 'song_id', how = 'left')

In [109]:
del [[members_df,songs_df]]
import gc
gc.collect()

3063

In [110]:
for i in range(0,8):
    all_data[i] = all_data[i].fillna(0)

In [111]:
all_data[0].value_counts()

0
465     5062629
458     1651779
921      610930
1609     450948
444      378968
         ...   
87            1
2015          1
1579          1
751           1
303           1
Name: count, Length: 166, dtype: int64

In [112]:
for i in range(0,8):
    all_data[i] = all_data[i].astype('category').cat.codes

In [113]:
all_data[0].value_counts()

0
119    5062629
118    1651779
155     610930
47      450948
116     378968
        ...   
149          1
65           1
44           1
138          1
102          1
Name: count, Length: 166, dtype: int64

In [114]:
nulls_cols = all_data.isnull().sum()
nulls_cols = nulls_cols[nulls_cols != 0]
list(nulls_cols.keys())

['song_length', 'artist_name', 'composer', 'lyricist', 'language']

In [115]:
for col in list(nulls_cols.keys()): 
    all_data[col] = all_data[col].fillna(all_data[col].mode()[0])

In [116]:
all_data.isnull().sum()

msno                  0
song_id               0
source_system_tab     0
source_screen_name    0
source_type           0
city                  0
bd                    0
gender                0
registered_via        0
reg_year              0
reg_month             0
reg_day               0
exp_year              0
exp_month             0
exp_day               0
song_length           0
artist_name           0
composer              0
lyricist              0
language              0
0                     0
1                     0
2                     0
3                     0
4                     0
5                     0
6                     0
7                     0
dtype: int64

In [117]:
all_data.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,city,bd,gender,registered_via,reg_year,...,lyricist,language,0,1,2,3,4,5,6,7
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,30,2,7,2012,...,,52.0,106,0,0,0,0,0,0,0
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,13,24,1,9,2011,...,,52.0,35,0,0,0,0,0,0,0
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,13,24,1,9,2011,...,,52.0,35,0,0,0,0,0,0,0
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,13,24,1,9,2011,...,,-1.0,5,0,0,0,0,0,0,0
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1,30,2,7,2012,...,,52.0,4,0,0,0,0,0,0,0


In [118]:
all_data[['composer']].nunique()

composer    86203
dtype: int64

In [119]:
categorical_col = ['city','gender','registered_via','language','artist_name','composer','lyricist']
for col in all_data.columns:
    if all_data[col].dtype in [bool ,object, 'category'] and len(all_data[col].unique()) <= 50:
        categorical_col.append(col)

In [120]:
categorical_col

['city',
 'gender',
 'registered_via',
 'language',
 'artist_name',
 'composer',
 'lyricist',
 'source_system_tab',
 'source_screen_name',
 'source_type']

In [121]:
for col in categorical_col:
    # Ensure the column is not of float16 type
    if all_data[col].dtype == 'float16':
        all_data[col] = all_data[col].astype('float32')  # or 'float64'

    all_data[col] = all_data[col].astype('category').cat.codes


In [122]:
all_data.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,city,bd,gender,registered_via,reg_year,...,lyricist,language,0,1,2,3,4,5,6,7
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,2,8,7,0,30,1,2,2012,...,0,8,106,0,0,0,0,0,0,0
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,4,9,5,11,24,0,3,2011,...,0,8,35,0,0,0,0,0,0,0
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,4,9,5,11,24,0,3,2011,...,0,8,35,0,0,0,0,0,0,0
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,4,9,5,11,24,0,3,2011,...,0,0,5,0,0,0,0,0,0,0
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,2,8,7,0,30,1,2,2012,...,0,8,4,0,0,0,0,0,0,0


In [123]:
all_data['msno'].value_counts()

msno
MXIMDXO0j3UpaT7FvOSGW6Y5zfhlh+xYjTqGoUdMzEE=    7894
o+5RNlSWrzvrphgBNGIo1FLkGxBgyICns6qXj3nS7Pk=    7318
FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=    7075
KGXNZ/H3VxvET/+rGxlrAe7Gpz2eKMXyuSg3xh8Ij1M=    6706
cqjRBV/jWN2ujhc+z/4tz+Mj6xEfflAAt6qBXCqxKvw=    5446
                                                ... 
U8+k0YmjY2aDIM7Wd9ejIF/XqZkoWbBlv+2booso86A=       1
0Yby35OBCWJyG0HOTIx003npPQ8mMY51ZI+Q6JulBqI=       1
YBw/FWdyTgpVStJFzPn0aVf3/dtQwxAiHmvAu3pxyoI=       1
g0cf2zdSgmLeBAHJZJZc85VD4l6f6xKkNegrh8VEAms=       1
q9mrx9lx8969hkkTf4h96PqKeeG0DTC5iMLuc/6VDSI=       1
Name: count, Length: 34403, dtype: int64

In [124]:
print('in',all_data.columns.map(type))
all_data.columns = all_data.columns.astype(str)
print('out',all_data.columns.map(type))

in Index([<class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>,
       <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>],
      dtype='object')
out Index([<class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>],
      dtype='object')


In [125]:
train_data = all_data[:ntrain].copy()
test_data = all_data[ntrain:]
train_data['target'] = y_train

train_data.shape , test_data.shape

((7377418, 29), (2556790, 28))

In [126]:
train_data['target'].head()

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: target, dtype: float16

In [127]:
# Check the data types of the 'target' column
print(train_data['target'].dtype)

# Convert the 'target' column to a different data type (e.g., int)
train_data['target'] = train_data['target'].astype(int)

# Now, you can proceed with the categorical conversion
train_data['target'] = train_data['target'].astype("category").cat.codes


float16


In [128]:
train_data['target'].head()

0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int8

In [129]:
X_train = train_data.drop(['target','msno','song_id'], axis = 1)
test_data = test_data.drop(['msno','song_id'],axis = 1)
y_train = train_data.target
X_train.shape , y_train.shape

((7377418, 26), (7377418,))

In [130]:
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer, PowerTransformer
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
test_data_scaled = scaler.transform(test_data)

In [131]:
X_train = X_train_scaled
test_data = test_data_scaled

In [ ]:
import lightgbm as lgb
start = time.time()
train_set = lgb.Dataset(X_train , y_train)
params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting': 'gbdt',
        'learning_rate': 0.3 ,
        'verbose': 0,
        'num_leaves': 216,
        'bagging_fraction': 0.95,
        'bagging_freq': 1,
        'bagging_seed': 1,
        'feature_fraction': 0.9,
        'feature_fraction_seed': 1,
        'max_bin': 510,
        'max_depth': 20,
        'num_rounds': 500,
        'metric' : 'auc'
    }

%time model_f1 = lgb.train(params, train_set=train_set,  valid_sets=train_set, verbose_eval=5)

end = time.time()
print(end - start)

/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.600462 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[5]	training's auc: 0.689416
[10]	training's auc: 0.700953
[15]	training's auc: 0.710045
[20]	training's auc: 0.716432
[25]	training's auc: 0.722384
[30]	training's auc: 0.726775
[35]	training's auc: 0.730963
[40]	training's auc: 0.734333
[45]	training's auc: 0.737629
[50]	training's auc: 0.740885
[55]	training's auc: 0.743809
[60]	training's auc: 0.746304
[65]	training's auc: 0.748518
[70]	training's auc: 0.750819
[75]	training's auc: 0.753333
[80]	training's auc: 0.755324
[85]	training's auc: 0.756772
[90]	training's auc: 0.758361
[95]	training's auc: 0.760253
[100]	training's auc: 0.761796
[105]	training's auc: 0.763141
[110]	training's auc: 0.764613
[115]	training's auc: 0.766564
[120]	training's auc: 0.76745
[125]	training's auc: 0.768466
[130]	training's 

In [ ]:
predictions = model_f1.predict(pd.DataFrame(test_data).values)
sub = pd.DataFrame()
sub['id'] = test_df['id']
sub['target'] = predictions
sub.to_csv('submission.csv',index=False)
sub.head()

### Technologies and Tools

   * Data Processing: Python, Pandas, NumPy, Scikit-Learn.
   * Machine Learning: TensorFlow, PyTorch, Keras.

### Challenges
  * Cold Start Problem: Difficulty in recommending tracks to new users with no listening history.
  * Diversity vs. Relevance: Balancing between recommending familiar tracks and introducing new, diverse music.
  * Scalability: Handling large volumes of data and real-time recommendations.
  * Bias and Fairness: Ensuring recommendations are fair and unbiased, considering diverse user backgrounds.


## Concluion

Music recommendation systems have transformed the way users discover and engage with music, significantly enhancing the listening experience. By leveraging advanced data science techniques, these systems provide personalized recommendations that cater to individual tastes and preferences. Despite challenges like the cold start problem and balancing diversity with relevance, continuous advancements in machine learning and data processing technologies are driving the evolution of more sophisticated and accurate recommendation models.

The future of music recommendation systems lies in further personalization, incorporating more nuanced user behavior analysis, and integrating cross-platform data to provide a holistic and seamless music discovery experience. As technology progresses, these systems will become more adept at understanding and predicting user preferences, ultimately creating a more enjoyable and immersive musical journey for listeners worldwide.